# Fitting RooFit-generated likelihoods

In this tutorial, we show how to create a negative log-likelihood function with the [RooFit framework](https://root.cern/manual/roofit/) and minimize it with iminuit. 

RooFit is a powerful fitting framework developed by CERN's ROOT team.
RooFit is very powerful and sophisticated, but there are a few reasons to use iminuit instead:

- RooFit documention is extensive, but lacking in important places
- RooFit interfaces are not Pythonic, they mimic the C++ interfaces (which are also dated)
- Errors are difficult to understand and debug since RooFit is internally executing C++ code
- You may experience segmentation faults when using RooFit from Python due to bugs in the ROOT Python layer (problems with handling life-times of dynamic C++ objects in Python correctly)

For these reasons, you may consider to transition to iminuit and its cost functions for your project. As a first step, you want to convince yourself that iminuit gives you the same fitting result as you get from RooFit. 

In [ ]:
# ROOT is best installed via a conda virtual environment from conda-forge
import ROOT

ModuleNotFoundError: No module named 'ROOT'

In [ ]:
# fix PRNG seed for RooFit random number generation
ROOT.RooRandom.randomGenerator().SetSeed(1)

We generate a Gaussian with mean 1 and width 3 and draw 10000 data points from it. 

In [ ]:
x = ROOT.RooRealVar("x", "x", -10, 10)
mean = ROOT.RooRealVar("mean", "mean of gaussian", 1, -10, 10)
sigma = ROOT.RooRealVar("sigma", "width of gaussian", 3, 0.1, 10)

gauss = ROOT.RooGaussian("gauss", "gaussian PDF", x, mean, sigma)

data = gauss.generate({x}, 10000)

We now fit this Gaussian. We use the `createNLL` method and a simple wrapper function `evaluate`. Note that this simple wrapping function does not propagate the parameter names of the Gaussian to iminuit. A future version of iminuit will come with a builtin wrapper that will also propagate the names and limits.

In [ ]:
from iminuit import Minuit

nll = gauss.createNLL(data)


def evaluate(*args):
    for par, arg in zip(nll.getVariables(), args):
        par.setVal(arg)
    # following RooMinimizerFcn.cxx
    nll.setHideOffset(False)
    r = nll.getVal()
    nll.setHideOffset(True)
    return r


evaluate.errordef = Minuit.LIKELIHOOD

m = Minuit(evaluate, *[p.getVal() for p in nll.getVariables()])
m.migrad()

Let's compare this to fitting directly with the `fitTo` method.

In [ ]:
gauss.fitTo(data);

The results are in agreement, because the results of a fit cannot depend on the minimizer. Technically, RooFit uses a different minimizer than iminuit by default. Unless you change some options, RooFit uses the original MINUIT Fortran implementation translated to C, while iminuit uses the rewritten Minuit2 C++ library.

Just doing the fitting with iminuit does not offer you a lot of advantages. Eventually, you want to switch completely. The equivalent of this exercise in pure Python is the following.

In [ ]:
from scipy.stats import truncnorm
from iminuit.cost import UnbinnedNLL
import numpy as np

xrange = (-10.0, 10.0)

rng = np.random.default_rng(1)
x = rng.normal(1, 3, size=10000)
x = x[(xrange[0] < x) & (x < xrange[1])]


def model(x, mu, sigma):
    zrange = np.subtract(xrange, mu) / sigma
    return truncnorm.pdf(x, *zrange, mu, sigma)


# better use numba_stats.truncnorm, which is simpler to use and faster
#
# from numba_stats import truncnorm
#
# def model(x, mu, sigma):
#     return truncnorm.pdf(x, *xrange, mu, sigma)

nll = UnbinnedNLL(x, model)
m = Minuit(nll, 1, 3)
m.migrad()

We do not get the exact same fitted values as before, since the data sample is different from the one generated by RooFit.

To get the exact same result, we need to convert the variable `data` which has the type `RooDataSet` into a numpy array. The ROOT Python layer offers the method `to_numpy` for this purpose.

In [ ]:
x = data.to_numpy()["x"]

nll = UnbinnedNLL(x, model)
m = Minuit(nll, 1, 3)
m.migrad()